In [1]:
import requests

def make_request(endpoint, payload=None):
    """
    Make a request to a specific endpoint on the weather API
    passing headers and optional payload.
    
    Parameters:
        - endpoint: The endpoint of the API you want to 
                    make a GET request to.
        - payload: A dictionary of data to pass along 
                   with the request.
    
    Returns:
        Response object.
    """
    return requests.get(
        f'https://www.ncdc.noaa.gov/cdo-web/api/v2/{endpoint}',
        headers={
            'token': 'PASTE_YOUR_TOKEN_HERE'
        },
        params=payload
    )

In [ ]:
import datetime

from IPython import display # for updating the cell dynamically

current = datetime.date(2022, 1, 1)
end = datetime.date(2023, 1, 1)

results = []

while current < end:
    # update the cell with status information
    display.clear_output(wait=True)
    display.display(f'Gathering data for {str(current)}')
    
    response = make_request(
        'data', 
        {
            'datasetid': 'GHCND', # Global Historical Climatology Network - Daily (GHCND) dataset
            'locationid': 'CITY:US360019', # NYC
            'startdate': current,
            'enddate': current,
            'units': 'metric',
            'limit': 1000 # max allowed
        }
    )

    if response.ok:
        # we extend the list instead of appending to avoid getting a nested list
        results.extend(response.json()['results'])

    # update the current date to avoid an infinite loop
    current += datetime.timedelta(days=1)
    

'Gathering data for 2022-01-01'

In [ ]:
import pandas as pd

df = pd.DataFrame(results)
df.head()

In [ ]:
df.to_csv('data/nyc_weather_2018.csv', index=False)


In [ ]:
import sqlite3

with sqlite3.connect('data/weather.db') as connection:
    df.to_sql(
        'weather', connection, index=False, if_exists='replace'
    )

In [ ]:
response = make_request(
    'stations', 
    {
        'datasetid': 'GHCND', # Global Historical Climatology Network - Daily (GHCND) dataset
        'locationid': 'CITY:US360019', # NYC
        'limit': 1000 # max allowed
    }
)

stations = pd.DataFrame(response.json()['results'])[['id', 'name', 'latitude', 'longitude', 'elevation']]
stations.to_csv('data/weather_stations.csv', index=False)

with sqlite3.connect('data/weather.db') as connection:
    stations.to_sql(
        'stations', connection, index=False, if_exists='replace'

### 

In [2]:
import pandas as pd

weather = pd.read_csv('Dataset/nyc_temperatures.csv')
weather.head()

,date,datatype,station,attributes,value
0,2018-10-01T00:00:00,TAVG,GHCND:USW00014732,"H,,S,",21.2
1,2018-10-01T00:00:00,TMAX,GHCND:USW00014732,",,W,2400",25.6
2,2018-10-01T00:00:00,TMIN,GHCND:USW00014732,",,W,2400",18.3
3,2018-10-02T00:00:00,TAVG,GHCND:USW00014732,"H,,S,",22.7
4,2018-10-02T00:00:00,TMAX,GHCND:USW00014732,",,W,2400",26.1


In [7]:
import numpy as np
from numpy import array
snow_data = weather.query('datatype == "SNOW" and value > 0 and station.str.contains("US1NY")')
snow_data.head()

TypeError: unhashable type: 'numpy.ndarray'

In [5]:
import sqlite3

with sqlite3.connect('data/weather.db') as connection:
    snow_data_from_db = pd.read_sql(
        'SELECT * FROM weather WHERE datatype == "SNOW" AND value > 0 and station LIKE "%US1NY%"', 
        connection
    )

snow_data.reset_index().drop(columns='index').equals(snow_data_from_db)

OperationalError: unable to open database file

In [8]:
SELECT * 
FROM weather 
WHERE datatype == "SNOW" AND value > 0 AND station LIKE "%US1NY%"

SyntaxError: invalid syntax (<ipython-input-8-fef842bad2b7>, line 1)